# Advent of Code 2023: coolcomputery

In [1]:
import numpy as np, math, itertools as it

In [2]:
def LINES():
    with open('in.txt') as f:
        return f.read().split('\n')

## Day 15

### 15a

In [8]:
toks=LINES()[0].split(',')
def H(s):
    out=0
    for ch in s:
        out=( (out+ord(ch))*17 ) % 256
    return out
sum(H(s) for s in toks)
# for s in toks:
#     print(s,H(s))

504449

### 15b

In [18]:
toks=LINES()[0].split(',')
def H(s):
    out=0
    for ch in s:
        out=( (out+ord(ch))*17 ) % 256
    return out

boxes=[[] for _ in range(256)]
for s in toks:
    label,action=None,None
    if s.find('=')>=0:
        label,action=s.split('=')
        action=int(action)
    else:
        assert s[-1]=='-'
        label,action=s[:-1],'-'
    # print(s,label,action,H(label))
    h=H(label)
    box=boxes[h]
    if action=='-':
        for i in range(len(box)-1,-1,-1):
            if box[i][0]==label:
                del box[i]
    else:
        done=False
        for i in range(len(box)):
            if box[i][0]==label:
                box[i]=(label,action)
                done=True
                break
        if not done:
            box.append((label,action))
#     display({i:box for i,box in enumerate(boxes) if len(box)>0})
sum((bi+1)*(i+1)*boxes[bi][i][1] for bi in range(len(boxes)) for i in range(len(boxes[bi])))

262044

## Day 14

### 14a

In [50]:
grid=LINES()
def rnorth(grid):
    R,C=len(grid),len(grid[0])
    out=[['.']*C for r in range(R)]
    for c in range(C):
        roll_i=0
        for r in range(R):
            if grid[r][c]=='#':
                out[r][c]='#'
                roll_i=r+1
            elif grid[r][c]=='O':
                out[roll_i][c]='O'
                roll_i+=1
    return out
def load(grid):
    R,C=len(grid),len(grid[0])
    return sum(
        R-r
        for r in range(R)
        for c in range(C)
        if grid[r][c]=='O'
    )
load(rnorth(grid))

113078

### 14b

In [75]:
grid=LINES()
def rnorth(grid):
    R,C=len(grid),len(grid[0])
    out=[['.']*C for r in range(R)]
    for c in range(C):
        roll_i=0
        for r in range(R):
            if grid[r][c]=='#':
                out[r][c]='#'
                roll_i=r+1
            elif grid[r][c]=='O':
                out[roll_i][c]='O'
                roll_i+=1
    return out
def rsouth(grid):
    return rnorth(grid[::-1])[::-1]
def rwest(grid):
    R,C=len(grid),len(grid[0])
    out=[['.']*C for r in range(R)]
    for r in range(R):
        roll_i=0
        for c in range(C):
            ch=grid[r][c]
            if ch=='#':
                out[r][c]='#'
                roll_i=c+1
            elif ch=='O':
                out[r][roll_i]='O'
                roll_i+=1
    return out
def rrev(grid):
    return [row[::-1] for row in grid]
def reast(grid):
    return rrev(rwest(rrev(grid)))
def spin(grid):
    return reast(rsouth(rwest(rnorth(grid))))
def load(grid):
    R,C=len(grid),len(grid[0])
    return sum(
        R-r
        for r in range(R)
        for c in range(C)
        if grid[r][c]=='O'
    )
def encode(grid):
    return '\n'.join(''.join(list(row)) for row in grid)

# display(grid)

hist={encode(grid):0}
M=grid
back_i=None
for _ in range(1000):
    M=spin(M)
    c=encode(M)
    if c in hist:
        back_i=hist[c]
        break
    hist[c]=len(hist)
#     display([''.join(row) for row in M])

'''
find t-th term (0-indexed) in sequence
0,1,...,h-1, b,b+1,...,h-1, b,b+1,...,h-1, ...
'''
def solve(h,b,t):
    print(h,b,t)
    if t<h:
        return t
    period=h-b
#     print(period)
    return ((t-h)%period)+b
ans_i=solve(len(hist),back_i,10**9)
hist_list={i:v for v,i in hist.items()}
def to_grid(code):
    return code.split('\n')
load(to_grid(hist_list[ans_i]))

117 103 1000000000


94255

## Day 13

### 13a

In [26]:
def is_refl(vs,i):
    if i<0 or i+1>=len(vs):
        return False
    l,r=i,i+1
    while l>=0 and r<len(vs):
        if vs[l]!=vs[r]:
            return False
        l-=1
        r+=1
    return True
def pts(rows):
    R,C=len(rows),len(rows[0])
    clms=[''.join(rows[r][c] for r in range(R)) for c in range(C)]
    ris=[r for r in range(R) if is_refl(rows,r)]
    cis=[c for c in range(C) if is_refl(clms,c)]
    assert len(ris)+len(cis)==1, (ris,cis)
#     print(ris,cis)
    return 100*(ris[0]+1) if len(ris)>0 else (cis[0]+1)

grids=[tok.split('\n') for tok in '\n'.join(LINES()).split('\n\n')]
grids

sum([pts(M) for M in grids])

33195

### 13b

In [44]:
def is_refl(vs,i):
    if i<0 or i+1>=len(vs):
        return False
    l,r=i,i+1
    while l>=0 and r<len(vs):
        if vs[l]!=vs[r]:
            return False
        l-=1
        r+=1
    return True
def refl_coords(rows):
    R,C=len(rows),len(rows[0])
    clms=[''.join(rows[r][c] for r in range(R)) for c in range(C)]
    ris=[r for r in range(R) if is_refl(rows,r)]
    cis=[c for c in range(C) if is_refl(clms,c)]
    return ris,cis
def smudge(rows):
    R,C=len(rows),len(rows[0])
    refls=refl_coords(rows)
    out=None
    b_unseen_refls=None
    for r in range(R):
        for c in range(C):
            M=[list(row) for row in rows]
            M[r][c]=('#' if M[r][c]=='.' else '.')
            M=[''.join(row) for row in M]
            nris,ncis=refl_coords(M)
            if len(set(nris)-set(refls[0]))>0 or len(set(ncis)-set(refls[1]))>0:
#                 print(r,c,refls,nris,ncis)
                unseens=(set(nris)-set(refls[0]),set(ncis)-set(refls[1]))
                if b_unseen_refls is None:
                    b_unseen_refls=unseens
                assert b_unseen_refls==unseens
                assert len(unseens[0])+len(unseens[1])==1
                out=list([100*(v+1) for v in unseens[0]]+[v+1 for v in unseens[1]])[0]
    assert out is not None
    return out

grids=[tok.split('\n') for tok in '\n'.join(LINES()).split('\n\n')]
grids

sum([smudge(M) for M in grids])

31836

## Day 12

### 12a

In [42]:
masks,runss=[],[]
for l in LINES():
    m,rs=l.split()
    masks.append(m)
    runss.append([int(v) for v in rs.split(',')])
masks,runss
def get_runs(s):
    c_='#'
    out=[]
    i=0
    while i<len(s):
        if s[i]==c_:
            j=i
            while j<len(s) and s[j]==c_:
                j+=1
            out.append(j-i)
            i=j
        else:
            i+=1
    return out
ans=0
for m,rs in zip(masks,runss):
    idxs=[c for c in range(len(m)) if m[c]=='?']
#     print(m,rs,idxs)
    cnt=0
    for states in it.product(['.','#'],repeat=len(idxs)):
        nm=list(m)
        for i,v in zip(idxs,states):
            nm[i]=v
        nm=''.join(nm)
#         print('\t',states,nm,get_runs(nm))
        if get_runs(nm)==rs:
            cnt+=1
#     print(cnt)
    ans+=cnt
ans

7792

### 12b

In [41]:
masks,runss=[],[]
for l in LINES():
    m,rs=l.split()
    masks.append(m)
    runss.append([int(v) for v in rs.split(',')])
masks,runss

C_='#'
def get_runs(s):
    out=[]
    i=0
    while i<len(s):
        if s[i]==C_:
            j=i
            while j<len(s) and s[j]==C_:
                j+=1
            out.append(j-i)
            i=j
        else:
            i+=1
    return out

def dp(M,runs):
    runs=tuple(runs)
    mem={}
    def solve(M,runs,ntrailing):
        out=None
        state=(M,runs,ntrailing)
        if state in mem:
            out=mem[state]
        else:
            if M=='':
                out=1 if runs==(tuple() if ntrailing==0 else (ntrailing,)) else 0
            elif M[-1]=='#':
                out=solve(M[:-1],runs,ntrailing+1)
            elif M[-1]=='.':
                if ntrailing==0:
                    out=solve(M[:-1],runs,0)
                else:
                    out=solve(M[:-1],runs[:-1],0) if len(runs)>0 and runs[-1]==ntrailing else 0
            else:
                assert M[-1]=='?'
                out=solve(M[:-1]+'.',runs,ntrailing)+solve(M[:-1]+'#',runs,ntrailing)
#             print('\t',M,runs,ntrailing,':',out)
        mem[state]=out
        return out
#     display(mem)
    return solve(M,runs,0)

ans=0
for m,rs in zip(masks,runss):
    cnt=dp('?'.join([m]*5),rs*5)
#     print(cnt)
    ans+=cnt
ans

13012052341533

## Day 11

### 11a

In [18]:
grid=LINES()
R,C=len(grid),len(grid[0])
pts=[(r,c) for r in range(R) for c in range(C) if grid[r][c]=='#']
ptrs,ptcs={r for r,c in pts},{c for r,c in pts}
rgaps,cgaps=[r for r in range(R) if r not in ptrs],[c for c in range(C) if c not in ptcs]
# display(rgaps,cgaps)
# xgrow,cgrow=[(1 if r in ptrs else 2) for r in range(R)],[(1 if c in ptcs else 2) for c in range(C)]
# xgrow,cgrow
def expand(A):
    A=set(A)
    A.add(0)
    A=sorted(A)
    out={}
    pad=0
    for i in range(len(A)):
        a=A[i]
        if i>0 and A[i]-A[i-1]>1:
            pad+=(A[i]-A[i-1])-1
        out[a]=a+pad
    return out
rM,cM=expand(ptrs),expand(ptcs)
npts=[(rM[r],cM[c]) for r,c in pts]
def dist(p0,p1):
    return sum(abs(p0[i]-p1[i]) for i in range(2))
sum(dist(p0,p1) for p0,p1 in it.combinations(npts,2))

9608724

### 11b

In [39]:
grid=LINES()
R,C=len(grid),len(grid[0])
pts=[(r,c) for r in range(R) for c in range(C) if grid[r][c]=='#']
ptrs,ptcs={r for r,c in pts},{c for r,c in pts}
rgaps,cgaps=[r for r in range(R) if r not in ptrs],[c for c in range(C) if c not in ptcs]
# display(rgaps,cgaps)
G=10**6
xgrow=np.array([(1 if r in ptrs else G) for r in range(R)])
cgrow=np.array([(1 if c in ptcs else G) for c in range(C)])
nxs,ncs=np.cumsum(xgrow)-1,np.cumsum(cgrow)-1
xgrow,cgrow,nxs,ncs
npts=[(nxs[r],ncs[c]) for r,c in pts]
# display(npts)
def dist(p0,p1):
    return sum(abs(p0[i]-p1[i]) for i in range(2))
sum(dist(p0,p1) for p0,p1 in it.combinations(npts,2))

904633799472

## Day 10

### 10a

In [44]:
grid=LINES()
dcoordss={
    '|':[(1,0),(-1,0)],
    '-':[(0,1),(0,-1)],
    'L':[(-1,0),(0,1)],
    'J':[(-1,0),(0,-1)],
    '7':[(1,0),(0,-1)],
    'F':[(1,0),(0,1)],
}
def grid2graph(grid):
    graph={}
    R,C=len(grid),len(grid[0])
    for r in range(R):
        for c in range(C):
            t=grid[r][c]
            neighbors=[]
            for dr,dc in dcoordss.get(t,[]):
                nr,nc=r+dr,c+dc
                if 0<=nr and nr<R and 0<=nc and nc<C:
                    neighbors.append((nr,nc))
            if len(neighbors)>0:
                graph[(r,c)]=neighbors
    return graph
G=grid2graph(grid)

R,C=len(grid),len(grid[0])
starts=[(r,c) for r in range(R) for c in range(C) if grid[r][c]=='S']
assert len(starts)==1
(sr,sc)=starts[0]
# display(G)
G[(sr,sc)]=[(r,c) for (r,c) in G if (sr,sc) in G[(r,c)]]
# display(G)

def bfs(G,v0):
    out=[v0]
    depths={v0:0}
    i=0
    while i<len(out):
        v=out[i]
        for n in G[v]:
            if n not in depths:
                depths[n]=depths[v]+1
                out.append(n)
        i+=1
    return depths
print(max(bfs(G,(sr,sc)).values()))

80


### 10b

In [75]:
grid=LINES()
dcoordss={
    '|':[(1,0),(-1,0)],
    '-':[(0,1),(0,-1)],
    'L':[(-1,0),(0,1)],
    'J':[(-1,0),(0,-1)],
    '7':[(1,0),(0,-1)],
    'F':[(1,0),(0,1)],
    '.':[(1,0),(-1,0),(0,1),(0,-1)]
}
def neighbors_of(R,C,r,c,dcs):
    out=[]
    for dr,dc in dcs:
        nr,nc=r+dr,c+dc
        if 0<=nr and nr<R and 0<=nc and nc<C:
            out.append((nr,nc))
    return out
def grid2graph(grid):
    graph={}
    R,C=len(grid),len(grid[0])
    for r in range(R):
        for c in range(C):
            t=grid[r][c]
            neighbors=[]
#             for dr,dc in dcoordss.get(t,[]):
#                 nr,nc=r+dr,c+dc
#                 if (0<=nr and nr<R and 0<=nc and nc<C
#                     and (t!='.' or grid[nr][nc]=='.')):
#                     neighbors.append((nr,nc))
            for nr,nc in neighbors_of(R,C,r,c,dcoordss.get(t,[])):
                if t!='.' or grid[nr][nc]=='.':
                    neighbors.append((nr,nc))
            if len(neighbors)>0:
                graph[(r,c)]=neighbors
    return graph
G=grid2graph(grid)

R,C=len(grid),len(grid[0])
starts=[(r,c) for r in range(R) for c in range(C) if grid[r][c]=='S']
assert len(starts)==1
(sr,sc)=starts[0]
# display(G)
G[(sr,sc)]=[(r,c) for (r,c) in G if (sr,sc) in G[(r,c)]]
start_typs=[typ for typ,dcs in dcoordss.items() if set(G[(sr,sc)])==set(neighbors_of(R,C,sr,sc,dcs))]
assert len(start_typs)
start_typ,=start_typs
print('start joint:',start_typ)
# display(G)

def dfs(G,v0):
    # max recursion depth exceeded
#     seen={v0}
#     seen_l=[v0]
#     def _dfs(v):
#         for n in G[v]:
#             if n not in seen:
#                 seen.add(n)
#                 seen_l.append(n)
#                 _dfs(n)
#     _dfs(v0)
#     return seen_l
    stack=[(v0,0)]
    out=[]
    seen={v0}
    while len(stack)>0:
        si=len(stack)-1
        v,ni=stack[si]
        if ni==0:
            out.append(v)
        if ni==len(G[v]):
            del stack[-1]
        else:
            n=G[v][ni]
            if n not in seen:
                seen.add(n)
                stack.append((n,0))
            ni+=1
            stack[si]=(v,ni)
    return out
loop=dfs(G,(sr,sc))

isolate=[
    [
        grid[r][c] if (r,c) in loop else '.'
        for c in range(C)
    ]
    for r in range(R)
]
isolate[sr][sc]=start_typ
# print('\n'.join(''.join(row) for row in isolate))
# print()
# print(loop)

#shoelace formula + Pick's theorem
def poly_area(coords):
    return sum(
        (y+ny)*(x-nx)
        for (x,y),(nx,ny) in zip(coords,coords[1:]+[coords[0]])
    )/2
print(abs(poly_area(loop))+1-len(loop)/2)

# def n_in(grid):
#     exteriors={}
#     R,C=len(grid),len(grid[0])
#     G=grid2graph(grid)
#     for r in range(R):
#         for c in range(C):
#             if r==0 or r==R-1 or c==0 or c==C-1:
#                 if (r,c) not in exteriors:
#                     exteriors.update(bfs(G,(r,c)))
# #     for r in range(R):
# #         for c in range(C):
# #             print((grid[r][c] if grid[r][c]!='.' else 'O' if (r,c) in exteriors else 'I'),end='')
# #         print()
#     return sum(
#         1
#         for r in range(R) for c in range(C)
#         if grid[r][c]=='.' and (r,c) not in exteriors
#     )
# def n_in(grid):
#     R,C=len(grid),len(grid[0])
#     for r in range(R):
#         cnt=0
#         for c in range(C):
#             cnt+=(0 if grid[r][c] in {'.','-'} else 1)
#             print((grid[r][c] if grid[r][c]!='.' else 'O' if cnt%2==0 else 'I'),end='')
#         print()
# n_in(isolate)

start joint: L
371.0


## Day 9

### 9a

In [113]:
seqs=[[int(v) for v in l.split()] for l in LINES()]

def solve(seq):
    diffss=[seq]
    H=seq
    while any(v!=0 for v in H):
        assert len(H)>0
        H=[H[i+1]-H[i] for i in range(len(H)-1)]
        diffss.append(H)
    # display(diffss)
    diffss[-1].append(0)
    for lvl in range(len(diffss)-2,-1,-1):
        diffss[lvl].append(diffss[lvl][-1]+diffss[lvl+1][-1])
    # display(diffss)
    return diffss[0][-1]
sum(solve(S) for S in seqs)

1861775706

### 9b

In [121]:
seqs=[[int(v) for v in l.split()] for l in LINES()]

def solve(seq):
    diffss=[seq]
    H=seq
    while any(v!=0 for v in H):
        assert len(H)>0
        H=[H[i+1]-H[i] for i in range(len(H)-1)]
        diffss.append(H)
    # display(diffss)
    diffss[-1].insert(0,0)
    for lvl in range(len(diffss)-2,-1,-1):
        diffss[lvl].insert(0,diffss[lvl][0]-diffss[lvl+1][0])
    # display(diffss)
    return diffss[0][0]
sum(solve(S) for S in seqs)

1082

## Day 8

### 8a

In [63]:
lines=LINES()
instr=lines[0]
ways={}
for l in lines[2:]:
    v,data=l.split(' = ')
    vl,vr=data[1:-1].split(', ')
    ways[v]=(vl,vr)

n=0
v='AAA'
while v!='ZZZ':
    d=instr[n%len(instr)]
    v=ways[v][0 if d=='L' else 1]
    n+=1
n

19783

### 8b

In [106]:
lines=LINES()
instr=lines[0]
ways={}
for l in lines[2:]:
    v,data=l.split(' = ')
    vl,vr=data[1:-1].split(', ')
    ways[v]=(vl,vr)

loops=[]
for v0 in ways:
    if v0[-1]=='A':
        start=(v0,0)
        seen={start:0}
        s=start
        iback=None
        while True:
            v,i=s
            d=instr[i]
            s=(ways[v][0 if d=='L' else 1],(i+1)%len(instr))
            if s in seen:
                iback=seen[s]
                break
            seen[s]=len(seen)
        seen_list={i:s for s,i in seen.items()}
        loops.append((
            seen_list,
            iback
        ))
'''
loop=(L,b)
locations: L[0],L[1],...,L[len(L)-1],L[b],L[b+1],...
    0<=i<len(L): at location L[i] at step i
    larger i: at location L[b + (i-len(L))%(len(L)-b)]
'''
# display(loops)

def loc_at_step(L,b,i):
    n=len(L)
    return L[i] if i<n else L[b + (i-n)%(n-b)]

for i in range(max(len(L) for L,b in loops)):
    # print([loc_at_step(L,b,i) for L,b in loops])
    if all(loc_at_step(L,b,i)[0][-1]=='Z' for L,b in loops):
        print(i)
        break

'''
find some i>len(L_k) for all (L_k,b_k) in loops
s.t. all L_k[b_k + (i-len(L_k))%(len(L_k)-b_k)] end with letter Z
'''
Z_idxs=[
    [idx for idx in range(len(L)) if L[idx][0][-1]=='Z']
    for L,b in loops
]
Z_idxs

def gcd(a,b):
    return gcd(b,a) if a>b else b if a==0 else gcd(a,b%a)
def congruence_pair(r1,m1, r2,m2):
    if m1<m2:
        return congruence_pair(r2,m2, r1,m1)
    r1%=m1
    r2%=m2
    M=(m1*m2)//gcd(m1,m2)
    # find k1 s.t. m1*k1+r1 ~ r2 mod m2
    for k1 in range(M//m1):
        i=m1*k1+r1
        if i%m2 == r2:
            return i,M
    return None

sol_i,sol_M=0,1
for zidx_list in it.product(*Z_idxs):
    print(zidx_list)
    for zidx,(L,b) in zip(zidx_list,loops):
        n=len(L)
        # need (i-n)%(n-b) + b == zidx
        # ==> i ~ (zidx-b)+n mod n-b
        # ==> i = (n-b)*m + (zidx-b) for some m
        print(n,b, f'need i%{n-b}={zidx-b+n}')
        sol_i,sol_M=congruence_pair(sol_i,sol_M, zidx-b+n,n-b)
sol_i,sol_M

(11653, 19783, 19241, 16531, 12737, 14363)
11657 4 need i%11653=23306
19785 2 need i%19783=39566
19245 4 need i%19241=38482
16535 4 need i%16531=33062
12743 6 need i%12737=25474
14365 2 need i%14363=28726


(0, 9177460370549)

## Day 7

### 7a

In [46]:
card2val={
    c:i+1
    for i,c in enumerate('123456789TJQKA')
}
hands,bids=[],{}
for l in LINES():
    h,b=l.split(' ')
    h=tuple(card2val[c] for c in h)
    hands.append(h)
    bids[h]=int(b)
assert len(set(hands))==len(hands)

def ctype(h):
    mset={}
    for c in h:
        mset[c]=mset.get(c,0)+1
    freqs=sorted(mset.values())
    if freqs[-1]==3:
        return 3.5 if len(freqs)>1 and freqs[-2]==2 else 3
    if freqs[-1]==2:
        return 2.5 if len(freqs)>1 and freqs[-2]==2 else 2
    return freqs[-1]

A=sorted(sorted(hands),key=ctype)
sum(bids[h]*(i+1) for i,h in enumerate(A))#,A

249483956

### 7b

In [55]:
card2val={
    c:i
    for i,c in enumerate('J123456789T.QKA')
}
hands,bids=[],{}
for l in LINES():
    h,b=l.split(' ')
    h=tuple(card2val[c] for c in h)
    hands.append(h)
    bids[h]=int(b)
assert len(set(hands))==len(hands)

def ctype(h):
    mset={}
    for c in h:
        if c!=0:
            mset[c]=mset.get(c,0)+1
    freqs=sorted(mset.values())
    if len(freqs)==0:
        freqs=[len(h)]
    else:
        freqs[-1]+=sum(1 for c in h if c==0)
#     print(h,freqs)
    if freqs[-1]==3:
        return 3.5 if len(freqs)>1 and freqs[-2]==2 else 3
    if freqs[-1]==2:
        return 2.5 if len(freqs)>1 and freqs[-2]==2 else 2
    return freqs[-1]

A=sorted(sorted(hands),key=ctype)
sum(bids[h]*(i+1) for i,h in enumerate(A))#,[(h,ctype(h)) for h in A]

252137472

## Day 6

### 6a

In [11]:
Ts,Ds=[
    [int(tok) for tok in l.split(':')[1].split()]
    for l in LINES()
]
out=1
for t,d in zip(Ts,Ds):
    nways=sum(1 for bt in range(t+1) if (t-bt)*bt>d)
    out*=nways
out

2612736

### 6b

In [17]:
T,D=[
    int(''.join(l.split(':')[1].split()))
    for l in LINES()
]
T,D

'''
(T-x)*x > D
x**2 - Tx < -D
x**2 - Tx + D < 0
(T +/- sqrt(T**2 - 4D)) / 2
'''
hi,lo=[(T+e*(T**2-4*D)**0.5)/2 for e in [1,-1]]
int(hi)-math.ceil(lo)+1

29891250

## Day 5

### 5a

In [19]:
sections='\n'.join(LINES()).split('\n\n')
seeds=[int(tok) for tok in sections[0].split(': ')[1].split(' ')]

def conversion(s):
    rng_strs=s.split(':\n')[1].split('\n')
#     out={}
#     for tok in rng_strs:
#         dest,src,le=(int(v) for v in tok.split())
#         for i in range(le):
#             assert src+i not in out
#             out[src+i]=dest+i
#     return out
    return [tuple(int(v) for v in tok.split()) for tok in rng_strs]
def convert(rngs,v):
    for dest,src,le in rngs:
        if src<=v and v<src+le:
            return dest+(v-src)
    return v

Ds=[conversion(sec) for sec in sections[1:]]
vals=seeds
for D in Ds:
    vals=[convert(D,v) for v in vals]
min(vals)

51580674

### 5b

In [34]:
sections='\n'.join(LINES()).split('\n\n')
seed_data=[int(tok) for tok in sections[0].split(': ')[1].split(' ')]

def conversion(s):
    rng_strs=s.split(':\n')[1].split('\n')
    out=[]
    for tok in rng_strs:
        dest,src,le=(int(v) for v in tok.split())
        out.append((src,src+le,dest-src))
    return out

Ds=[conversion(sec) for sec in sections[1:]]
seed_rngs=[(src,src+le) for src,le in zip(seed_data[::2],seed_data[1::2])]

# if x in range (dest,src,le): x -> x+(dest-src)
#  src <= x < src+le

def convert_range(start,converters):
#     print(start,converters)
    # assume that converters operate over disjoint ranges
    converters=sorted(converters,key=lambda rng:rng[1])
    _clo,_chi=converters[0][0],converters[-1][1]
    _lo,_hi=min(start[0],_clo),max(start[1],_chi)
    converters=[(_lo,_clo,0)]+converters+[(_chi,_hi,0)]
    id_rngs=[
        (h0,l1,0)
        for (l0,h0,sh0),(l1,h1,sh1) in zip(converters,converters[1:])
    ]
    converters=sorted(converters+id_rngs,key=lambda rng:rng[1])
#     print(converters)
    out=[]
    for clo,chi,sh in converters:
        alo,ahi=max(start[0],clo),min(start[1],chi)
#         print(f'\t{[alo,ahi]}')
        if alo<ahi:
            out.append(
                (alo+sh,ahi+sh)
            )
    return out

rngs=seed_rngs
for D in Ds:
    rngs=[nrng for rng in rngs for nrng in convert_range(rng,D)]
#     print(rngs)
min(lo for lo,hi in rngs)#,rngs

99751240

## Day 4

### 4a

In [14]:
cards=[
    tuple(
        [int(v) for v in tok.split()]
        for tok in l.split(': ')[1].split(' | ')
    )
    for l in LINES()
]
# display(cards)
out=0
for wins,yours in cards:
    cnt=sum(1 for v in yours if v in wins)
    out+=2**(cnt - 1) if cnt>0 else 0
out

17782

### 4b

In [17]:
cards=[
    tuple(
        [int(v) for v in tok.split()]
        for tok in l.split(': ')[1].split(' | ')
    )
    for l in LINES()
]
N=len(cards)
mults=[1]*N
for i,(wins,yours) in enumerate(cards):
    cnt=sum(1 for v in yours if v in wins)
    for k in range(cnt):
        mults[i+k+1]+=mults[i]
sum(mults)

8477787

## Day 3

### 3a

In [10]:
grid=LINES()
# print(grid)
R,C=len(grid),len(grid[0])
adj=[[False]*C for r in range(R)]
dirs=[
    (1,0),(1,1),(0,1),(-1,1),(-1,0),(-1,-1),(0,-1),(1,-1)
]
for r in range(R):
    for c in range(C):
        s=grid[r][c]
        if s not in '.1234567890':
            for dr,dc in dirs:
                nr,nc=r+dr,c+dc
                if 0<=nr and nr<R and 0<=nc and nc<C:
                    adj[nr][nc]=True

good_nums=[]
for r in range(R):
    j=0
    while j<C:
        if grid[r][j] in '1234567890':
            k=j
            while k<C and grid[r][k] in '1234567890':
                k+=1
#             print(j,k,grid[r][j:k])
            if any(adj[r][i] for i in range(j,k)):
                good_nums.append(int(grid[r][j:k]))
            j=k
        else:
            j+=1
# print(good_nums)
sum(good_nums)

537832

### 3b

In [18]:
grid=LINES()
# print(grid)
R,C=len(grid),len(grid[0])
num_idx=[[None]*C for r in range(R)]
num_vals={}
for r in range(R):
    j=0
    while j<C:
        if grid[r][j] in '1234567890':
            k=j
            while k<C and grid[r][k] in '1234567890':
                k+=1
            n_id=len(num_vals)
            n_v=int(grid[r][j:k])
            num_vals[n_id]=n_v
            for i in range(j,k):
                num_idx[r][i]=n_id
            j=k
        else:
            j+=1

# display(num_idx)

dirs=[
    (1,0),(1,1),(0,1),(-1,1),(-1,0),(-1,-1),(0,-1),(1,-1)
]
rats=[]
for r in range(R):
    for c in range(C):
        s=grid[r][c]
        if s=='*':
            n_ids=set()
            for dr,dc in dirs:
                nr,nc=r+dr,c+dc
                if 0<=nr and nr<R and 0<=nc and nc<C:
                    n_id=num_idx[nr][nc]
                    if n_id is not None:
                        n_ids.add(n_id)
            if len(n_ids)==2:
                rats.append(math.prod(num_vals[n_id] for n_id in n_ids))
sum(rats)#,rats

81939900

## Day 2

### 2a

In [59]:
games=[s[s.find(' '):] for s in LINES()]
allow={'red':12, 'green':13, 'blue':14}
out=0
for G in games:
    gi,G=G.split(': ')
    gi=int(gi)
    G=G.split('; ')
#     print(gi,G)
    ssets=[]
    for s in G:
        D={}
        for tok in s.split(', '):
            cnt,col=tok.split(' ')
            D[col]=int(cnt)
        ssets.append(D)
    bnd={}
    for D in ssets:
        for col,v in D.items():
            bnd[col]=max(bnd.get(col,0),v)
    if all(bnd[c]<=allow[c] for c in allow):
#         print(bnd)
        out+=gi
out

2632

### 2b

In [60]:
games=[s[s.find(' '):] for s in LINES()]
allow={'red':12, 'green':13, 'blue':14}
out=0
for G in games:
    gi,G=G.split(': ')
    gi=int(gi)
    G=G.split('; ')
#     print(gi,G)
    ssets=[]
    for s in G:
        D={}
        for tok in s.split(', '):
            cnt,col=tok.split(' ')
            D[col]=int(cnt)
        ssets.append(D)
    bnd={}
    for D in ssets:
        for col,v in D.items():
            bnd[col]=max(bnd.get(col,0),v)
#     if all(bnd[c]<=allow[c] for c in allow):
#     print(bnd)
    out+=math.prod(bnd.values())
out

69629

## Day 1

In [26]:
with open('in.txt') as f:
    lines=f.read().split('\n')

### 1a

In [27]:
out=0
for l in lines:
    digs=[int(c) for c in l if c in '1234567890']
    out+=digs[0]*10+digs[-1]
out

55621

### 1b

In [28]:
D={s:i for i,s in enumerate('zero one two three four five six seven eight nine'.split())}
out=0
for l in lines:
    digs=[]
    for i in range(len(l)):
        if l[i] in '1234567890':
            digs.append(int(l[i]))
        else:
            for s,v in D.items():
                if l[i:].startswith(s):
                    digs.append(v)
    x=digs[0]*10+digs[-1]
#     print(l,x)
    out+=x
out

53592